In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import pandas as pd
import plotly.express as px
from crud import AnimalShelter  # Import your CRUD module here

# Set up MongoDB connection
username = "aacuser"
password = "CutMeinfor20%!"
host = "nv-desktop-services.apporto.com"
port = 30960
db_name = "AAC"
collection_name = "animals"
shelter = AnimalShelter(username, password, host, port, db_name, collection_name)

# Initialize Dash app
app = JupyterDash(__name__)

# Initial data load (unfiltered)
df = pd.DataFrame.from_records(shelter.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Define layout
app.layout = html.Div([
    html.A(html.Img(src='/assets/grazioso_logo.png', id='logo', style={'height':'100px', 'width':'auto'}), href='http://www.snhu.edu'),  # Add logo with link
    html.H1("Grazioso Salvare Dashboard - Your Name"),
    dcc.RadioItems(
        id='rescue-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',
        labelStyle={'display': 'inline-block'}
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        page_size=10
    ),
    html.Br(),
    html.Div([
        # Div for map
        html.Div(id='map-id', style={'width': '49%', 'display': 'inline-block', 'vertical-align': 'top'}),
        # Div for pie chart
        html.Div(dcc.Graph(id='second-chart'), style={'width': '49%', 'display': 'inline-block', 'vertical-align': 'top'}),
    ])
])

# Callback for filtering data and updating widgets
@app.callback(
    [Output('datatable-id', 'data'),
     Output('map-id', 'children'),
     Output('second-chart', 'figure')],
    [Input('rescue-type', 'value')]
)
def update_dashboard(rescue_type):
    # Define queries based on the selected rescue type
    query = {}
    if rescue_type == 'water':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                 "sex_upon_outcome": "Intact Female",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
    elif rescue_type == 'mountain':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
    elif rescue_type == 'disaster':
        query = {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}

    data = pd.DataFrame.from_records(shelter.read(query))
    if '_id' in data.columns:
        data.drop(columns=['_id'], inplace=True)

    # Update data table
    table_data = data.to_dict('records')
    
    # Update map
    map_children = [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Add markers for each row
            *[
                dl.Marker(position=[row['location_lat'], row['location_long']],
                          children=[
                              dl.Tooltip(row['breed']),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(f"Breed: {row['breed']}"),
                                  html.P(f"Age: {row['age_upon_outcome']}"),
                                  html.P(f"Sex: {row['sex_upon_outcome']}")
                              ])
                          ]) for i, row in data.iterrows()
            ]
        ])
    ]
    
    # Update second chart (e.g., pie chart)
    fig = px.pie(data, names='breed', title="Breed Distribution")

    return table_data, map_children, fig

# Run the app
app.run_server(debug=True, port=8051)


Dash app running on http://127.0.0.1:8051/
